## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-19-01-17-19 +0000,nypost,Sheriff probing Lake Tahoe guide company after...,https://nypost.com/2026/02/18/us-news/sheriff-...
1,2026-02-19-01-16-10 +0000,bbc,Etsy sells second-hand fashion app Depop to eB...,https://www.bbc.com/news/articles/cx240kme2k8o...
2,2026-02-19-01-13-48 +0000,nyt,What to Know About the Sierra Nevada Avalanche,https://www.nytimes.com/2026/02/18/us/avalanch...
3,2026-02-19-01-13-45 +0000,nypost,Delta plane forced to return to Houston airpor...,https://nypost.com/2026/02/18/us-news/delta-pl...
4,2026-02-19-01-13-18 +0000,nyt,Israel’s Actions Questioned at Security Counci...,https://www.nytimes.com/2026/02/18/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2503/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
113,trump,32
60,new,22
216,california,18
187,up,13
7,avalanche,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
233,2026-02-18-15-33-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...,109
303,2026-02-18-11-00-00 +0000,nypost,Lara Trump reveals Trump’s unexpected reaction...,https://nypost.com/2026/02/18/us-news/lara-tru...,90
284,2026-02-18-11-59-00 +0000,wsj,Stephen Colbert’s stinging criticism of Presid...,https://www.wsj.com/politics/how-stephen-colbe...,88
161,2026-02-18-19-06-00 +0000,wsj,NIH Director Dr. Jay Bhattacharya will step in...,https://www.wsj.com/politics/policy/rfk-jr-cdc...,86
52,2026-02-18-23-28-39 +0000,nypost,City Hall backtracks on claim that 200 new att...,https://nypost.com/2026/02/18/us-news/city-bac...,83


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
233,109,2026-02-18-15-33-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...
257,69,2026-02-18-14-18-49 +0000,latimes,Eight skiers dead and one missing in deadliest...,https://www.latimes.com/california/story/2026-...
284,56,2026-02-18-11-59-00 +0000,wsj,Stephen Colbert’s stinging criticism of Presid...,https://www.wsj.com/politics/how-stephen-colbe...
10,53,2026-02-19-00-55-16 +0000,bbc,Tech firms will have 48 hours to remove abusiv...,https://www.bbc.com/news/articles/cz6ed1549yvo...
151,52,2026-02-18-19-43-00 +0000,wsj,Fed officials signaled little appetite for red...,https://www.wsj.com/economy/central-banking/fe...
274,42,2026-02-18-12-56-39 +0000,cbc,No sign of a breakthrough in Russia-Ukraine ta...,https://www.cbc.ca/news/world/switzerland-russ...
149,41,2026-02-18-19-53-13 +0000,nypost,A-list star’s energy drink brand may owe you u...,https://nypost.com/2026/02/18/lifestyle/dwayne...
42,41,2026-02-18-23-43-53 +0000,nypost,Trans Rhode Island hockey killer’s daughter de...,https://nypost.com/2026/02/18/us-news/trans-rh...
343,37,2026-02-18-04-04-52 +0000,latimes,"UCLA fires top finance officer, saying he made...",https://www.latimes.com/california/story/2026-...
225,33,2026-02-18-16-06-31 +0000,nypost,21-year-old Israeli soldier killed by friendly...,https://nypost.com/2026/02/18/world-news/israe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
